In [1]:
import os
from dotenv import load_dotenv
from langchain_neo4j import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chains import RetrievalQA


import vansy_f8a_genai_test.report_a03_rag_and_reasoning_frameworks_tutorial_prompt as prompt_hub

In [2]:
load_dotenv()

True

In [3]:
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_18391/88119791.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


### Hospital Cypher Chain

In [4]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

graph.refresh_schema()

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_18391/249957495.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
cypher_generation_template = prompt_hub.CYPHER_GENERATION_TEMPLATE
qa_generation_template = prompt_hub.QA_GENERATION_TEMPLATE

In [6]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"], template=cypher_generation_template
)

qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)


In [7]:
hospital_cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    qa_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    graph=graph,
    verbose=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    validate_cypher=True,
    top_k=100,
    allow_dangerous_requests=True,
    # use_function_response=True
)

### Hospital Review Chain

In [8]:
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [9]:
neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="reviews",
    node_label="Review",
    text_node_properties=[
        "physician_name",
        "patient_name",
        "text",
        "hospital_name",
    ],
    embedding_node_property="embedding",
)

In [10]:
review_generation_template = prompt_hub.REVIEW_GENERATION_TEMPLATE

In [11]:
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_generation_template
    )
)

In [12]:
review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)

In [13]:
messages = [review_system_prompt, review_human_prompt]
review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

In [14]:
reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    chain_type="stuff",
    retriever=neo4j_vector_index.as_retriever(k=12),
)
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt

### ReAct Agent

In [15]:
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_openai import ChatOpenAI
from langchain import hub

In [16]:
# 1. Define the tools for agent can use
tools = [
    Tool(
    name="HospitalGraphQA",
    # The 'func' is the chain's invoke method. We use a lambda to map the
    # string input from the agent to the dictionary input the chain expects.
    func=lambda q: hospital_cypher_chain.invoke({"query": q}),
    description="""Useful for answering questions about patients,
        physicians, hospitals, insurance payers, patient review
        statistics, and hospital visit details. Use the entire prompt as
        input to the tool. For instance, if the prompt is 'How many visits
        have there been?", the input should be "How many visits have
        there been?'
        """
    ),
    Tool(
    name="HospitalReviewsQA",
    func=lambda q: reviews_vector_chain.invoke({"query": q}),
    description="""Useful when you need to answer questions
        about patient experiences, feelings, or any other qualitative
        question that could be answered about a patient using semantic
        search. Not useful for answering objective questions that involve
        counting, percentages, aggregations, or listing facts. Use the
        entire prompt as input to the tool. For instance, if the prompt is
        "Are patients satisfied with their care?", the input should be
        "Are patients satisfied with their care?".
        """
    )
]

In [17]:
# 2. Set up the ReAct Agent
# Pull the standard ReAct prompt
prompt = hub.pull("hwchase17/react")

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
prompt.dict()

{'name': None,
 'input_variables': ['agent_scratchpad', 'input', 'tool_names', 'tools'],
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'react',
  'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'},
 'tags': None,
 'template': 'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',
 'template_format': 'f-string',
 'validate_t

In [19]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
custom_prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

PREVIOUS CHAT HISTORY:
{chat_history}

NEW QUESTION:
Question: {input}
Thought:{agent_scratchpad}
"""

In [21]:
prompt = PromptTemplate(
    input_variables=['chat_history','agent_scratchpad', 'input', 'tool_names', 'tools'],
    template=custom_prompt_template,
)

In [22]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [23]:
# Create the agent
agent = create_react_agent(agent_llm, tools, prompt)

In [27]:
# Create the agent executor to run the agent
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, verbose=True, 
                               memory=memory, 
                               return_intermediate_steps=True,
                               handle_parsing_errors=True)

In [32]:
# --- 3. The Interactive Chat Loop ---
print("Chat with the Hospital AI Agent! Type 'exit' to quit.")
while True:
    try:
        user_question = input("You: ")
        print(f"\nUser: {user_question}")
        if user_question.lower() in ["exit", "quit"]:
            print("\nAgent: Goodbye!")
            break
        
        # Invoke the agent executor with the user's question
        response = agent_executor.invoke({"input": user_question})
        
        # Print the final answer
        print(f"\nAgent: {response['output']}")

    except Exception as e:
        print(f"An error occurred: {e}")

Chat with the Hospital AI Agent! Type 'exit' to quit.

User: how many hospital in Florida?


> Entering new AgentExecutor chain...
I need to find out the number of hospitals in Florida. Since this is an objective question that involves counting, I will use the HospitalGraphQA tool to get the information.

Action: HospitalGraphQA  
Action Input: how many hospital in Florida?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'FL'
RETURN COUNT(h) AS hospital_count
Full Context:
[{'hospital_count': 7}]

> Finished chain.
{'query': 'how many hospital in Florida?', 'result': 'There are 7 hospitals in Florida.'}

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


I now know the final answer.  
Final Answer: There are 7 hospitals in Florida.

> Finished chain.

Agent: There are 7 hospitals in Florida.

User: can you name them?


> Entering new AgentExecutor chain...
I need to find out the names of the hospitals in Florida. Since this is an objective question that involves listing facts, I will use the HospitalGraphQA tool to retrieve this information.

Action: HospitalGraphQA  
Action Input: "Can you name the hospitals in Florida?"  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'FL'
RETURN h.name AS hospital_name
Full Context:
[{'hospital_name': 'Walton LLC'}, {'hospital_name': 'Wheeler, Bryant and Johns'}, {'hospital_name': 'Brown Inc'}, {'hospital_name': 'Smith, Edwards and Obrien'}, {'hospital_name': 'Rose Inc'}, {'hospital_name': 'Jordan Inc'}, {'hospital_name': 'Lewis-Nelson'}]

> Finished chain.
{'query': 'Can you name the hospitals in Florida?', 'result': 'The hospitals in Florida a

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


I now know the final answer.  
Final Answer: The hospitals in Florida are as follows:  
1. Walton LLC  
2. Wheeler, Bryant and Johns  
3. Brown Inc  
4. Smith, Edwards and Obrien  
5. Rose Inc  
6. Jordan Inc  
7. Lewis-Nelson  

> Finished chain.

Agent: The hospitals in Florida are as follows:  
1. Walton LLC  
2. Wheeler, Bryant and Johns  
3. Brown Inc  
4. Smith, Edwards and Obrien  
5. Rose Inc  
6. Jordan Inc  
7. Lewis-Nelson

User: exit

Agent: Goodbye!


### Evaluation

In [28]:
from agent_evaluator import evaluate_agent
import pandas as pd

In [29]:
csv_filepath = "../data/evaluation_data.csv"

In [30]:
summary, details_df = evaluate_agent(agent_executor, csv_filepath)

100
Starting evaluation on 100 questions...

[Test 1/100] Question: What is the total number of patients treated at Grace Memorial Hospital in May 2023?


> Entering new AgentExecutor chain...
To find the total number of patients treated at Grace Memorial Hospital in May 2023, I will need to use the HospitalGraphQA tool, as it can provide objective data regarding patient counts.

Action: HospitalGraphQA  
Action Input: What is the total number of patients treated at Grace Memorial Hospital in May 2023?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)<-[:AT]-(v:Visit)<-[:HAS]-(p:Patient)
WHERE h.name = 'Grace Memorial Hospital' AND v.admission_date >= '2023-05-01' AND v.admission_date < '2023-06-01'
RETURN COUNT(p) AS total_patients_treated
Full Context:
[{'total_patients_treated': 0}]

> Finished chain.
{'query': 'What is the total number of patients treated at Grace Memorial Hospital in May 2023?', 'result': 'In May 2023, a total of 0 patients were tr

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


To find the list of physicians specializing in Cardiology, I will use the HospitalGraphQA tool, as it can provide factual information about physicians.

Action: HospitalGraphQA  
Action Input: List all physicians specializing in Cardiology.  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (phy:Physician)
WHERE phy.school CONTAINS 'Cardiology'
RETURN phy.name AS physician_name
Full Context:
[]

> Finished chain.
{'query': 'List all physicians specializing in Cardiology.', 'result': "I don't know the answer."}It seems that the HospitalGraphQA tool did not return any information regarding the physicians specializing in Cardiology. This could mean that there are no records available or that the query did not yield results. 

I will try to gather more qualitative information about the presence of cardiologists or patient experiences related to cardiology. 

Action: HospitalReviewsQA  
Action Input: Are there any cardiologists at the hospital?  {'query': 'Are there any c

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


To find the billing amount for visit ID 789123, I need to query the relevant data about hospital visits, which includes billing information.

Action: HospitalGraphQA  
Action Input: What is the billing amount for visit ID 789123?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visit)-[c:COVERED_BY]->(p:Payer)
WHERE v.id = 789123
RETURN c.billing_amount AS billing_amount
Full Context:
[]

> Finished chain.
{'query': 'What is the billing amount for visit ID 789123?', 'result': "I'm sorry, but I don't know the answer to your question about the billing amount for visit ID 789123."}I was unable to retrieve the billing amount for visit ID 789123, as the information is not available. 

Final Answer: I do not have the billing amount for visit ID 789123.

> Finished chain.
  -> Expected Tool: HospitalGraphQA
  -> Actual Tool:   HospitalGraphQA
  -> Selected At:   2025-06-18 04:30:56
  -> Correct:       True
  -> Latency:       5.48s

[Test 4/100] Question: Which insuran

...

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


To find out which other hospitals Alexander Jimenez is affiliated with, I will need to query the relevant database for information about his affiliations. 

Action: HospitalGraphQA  
Action Input: "What other hospitals is Alexander Jimenez affiliated with?"  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (phy:Physician {name: 'Alexander Jimenez'})<-[:EMPLOYS]-(h:Hospital)
WITH h AS hospital
MATCH (h2:Hospital)-[:EMPLOYS]->(phy)
WHERE h2 <> hospital
RETURN h2.name AS affiliated_hospital
Full Context:
[{'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin and Cooper'}, {'affiliated_hospital': 'Burke, Griffin 

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


Action: None  
Action Input: None  None is not a valid tool, try one of [HospitalGraphQA, HospitalReviewsQA].Action: None  
Action Input: None  None is not a valid tool, try one of [HospitalGraphQA, HospitalReviewsQA].I don't have a specific question to answer or action to take based on the input provided. It seems like the user is concluding the conversation. 

Final Answer: No further action is needed at this time.

> Finished chain.
  -> Expected Tool: no
  -> Actual Tool:   no
  -> Selected At:   2025-06-18 04:39:09
  -> Correct:       True
  -> Latency:       2.53s

--- Evaluation Complete ---


/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


In [33]:
if summary:
    print("\n\n📊 EVALUATION SUMMARY:")
    for key, value in summary.items():
        if isinstance(value, float):
            print(f"- {key.replace('_', ' ').title()}: {value:.4f}")
        else:
            print(f"- {key.replace('_', ' ').title()}: {value}")

    print("\n\n📋 DETAILED RESULTS (PREVIEW):")
    display(details_df)

    # NEW: Save the detailed results DataFrame to a new CSV file
    output_filename = "../data/evaluation_results.csv"
    details_df.to_csv(output_filename, index=False)
    print(f"\n\n💾 Detailed results saved to '{output_filename}'")



📊 EVALUATION SUMMARY:
- Total Questions: 100
- Total Cost Usd: 0.0131
- Total Tokens: 95416
- Average Latency Sec: 5.1815
- Tool Selection Accuracy Percent: 90.7216


📋 DETAILED RESULTS (PREVIEW):


,num,question,expected_tool,actual_tool,tool_selection_datetime,is_correct,latency_sec,final_answer
0,0,What is the total number of patients treated a...,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:30:43,True,5.146701,"In May 2023, a total of 0 patients were treate..."
1,1,List all physicians specializing in Cardiology.,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:30:50,True,7.507440,I do not have sufficient information to provid...
2,2,What is the billing amount for visit ID 789123?,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:30:56,True,5.484712,I do not have the billing amount for visit ID ...
3,3,Which insurance provider is associated with pa...,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:31:01,True,4.843179,I do not have information about the insurance ...
4,5,Find all patients treated by Dr. Ben Adams.,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:31:09,True,4.431775,I don't have the information about the patient...
...,...,...,...,...,...,...,...,...
92,95,Now show me the total number of visits for tha...,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:38:45,True,4.383787,The total number of visits for Downtown Clinic...
93,96,Who is patient Jane Doe's primary care physician?,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:38:49,True,4.381363,I don't have information on Jane Doe's primary...
94,97,What do other patients say about him?,HospitalReviewsQA,HospitalReviewsQA,2025-06-18 04:38:54,True,5.372438,I don't have specific reviews about the physic...
95,98,And what other hospitals is he affiliated with?,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:39:07,True,12.240768,"Alexander Jimenez is affiliated with Burke, Gr..."




💾 Detailed results saved to '../data/evaluation_results.csv'


In [32]:
details_df.dtypes

num                          int64
question                    object
expected_tool               object
actual_tool                 object
tool_selection_datetime     object
is_correct                    bool
latency_sec                float64
final_answer                object
dtype: object